<a href="https://colab.research.google.com/github/MassimilianoBiancucci/Tensorflow-exercises/blob/master/CNNs/Training_a_CNN_on_MINST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a CNN on MINST dataset

In this section we download the packets needed for run a tensorboard remotly on colab and get a link at ngrok.com to access to the tensorboard trought the colab's firewall.
![alt text](https://gitcdn.xyz/cdn/Tony607/blog_statics/d425c3fe4cf0d92067572e25ae6cc3198d51936b//images/ngrok/ngrok.jpg)

In [1]:
!rm /content/ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-04-23 19:59:47--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.204.136.9, 34.206.253.53, 54.174.228.92, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.204.136.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.16’

ngrok-stable-linux- 100%[===================>]  14.28M  58.3MB/s    in 0.2s    

2019-04-23 19:59:48 (58.3 MB/s) - ‘ngrok-stable-linux-amd64.zip.16’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


here we launch the tensorboard in background and  set the directory for saving session log.


In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

Then, we can run ngrok to tunnel TensorBoard port 6006 to the outside world. This command also runs in the background.


In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

Now we get the public URL where we can access the colab TensorBoard.
It's important keep in mind that the training have to start before you can see somthing in. 


In [4]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://d0faa220.ngrok.io


In [0]:
!rm -r ./log/train
!rm -r ./log/test

In [6]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

input_size = 784
no_classes = 10
batch_size = 100
total_batches = 200
iter_no = 30

In [0]:
x_input = tf.placeholder(tf.float32, shape=[None, input_size])
y_input = tf.placeholder(tf.float32, shape=[None, no_classes])
keep_prob = tf.placeholder(tf.float32) #dropuot keep_prob placeholder

x_input_reshape = tf.reshape(x_input, [-1, 28, 28, 1], name='input_reshape')

In [0]:
def add_variable_summary(tf_variable, summary_name):
  with tf.name_scope(summary_name + '_summary'):
    mean = tf.reduce_mean(tf_variable)
    tf.summary.scalar('Mean', mean)
    with tf.name_scope('standard_deviation'):
        standard_deviation = tf.sqrt(tf.reduce_mean(
            tf.square(tf_variable - mean)))
    tf.summary.scalar('StandardDeviation', standard_deviation)
    tf.summary.scalar('Maximum', tf.reduce_max(tf_variable))
    tf.summary.scalar('Minimum', tf.reduce_min(tf_variable))
    tf.summary.histogram('Histogram', tf_variable)

In [0]:
def convolution_layer(input_layer, filters, kernel_size=[5, 5], activation=tf.nn.relu):
    with tf.device('/gpu:0'):  
        layer = tf.layers.conv2d(
            inputs=input_layer,
            filters=filters,
            kernel_size=kernel_size,
            activation=activation
        )
    add_variable_summary(layer, 'convolution')
    return layer

In [0]:
def pooling_layer(input_layer, pool_size=[2, 2], strides=2):
    with tf.device('/gpu:0'):  
      layer = tf.layers.max_pooling2d(
          inputs=input_layer,
          pool_size=pool_size,
          strides=strides
      )
    add_variable_summary(layer, 'pooling')
    return layer

In [0]:
def dense_layer(input_layer, units, activation=tf.nn.relu):
   with tf.device('/gpu:0'):    
      layer = tf.layers.dense(
                                inputs=input_layer,
                                units=units,
                                activation=activation
                             )
   add_variable_summary(layer, 'dense')
   return layer


In [0]:
convolution_layer_1 = convolution_layer(x_input_reshape, 64)
pooling_layer_1 = pooling_layer(convolution_layer_1)
convolution_layer_2 = convolution_layer(pooling_layer_1, 128)
pooling_layer_2 = pooling_layer(convolution_layer_2)
flattened_pool = tf.reshape(pooling_layer_2, [-1, 4 * 4 * 128],
                            name='flattened_pool')
dense_layer_bottleneck = dense_layer(flattened_pool, 1024)
with tf.device('/gpu:0'):
    dropout_layer = tf.layers.dropout(
            inputs=dense_layer_bottleneck,
            rate = 1 - keep_prob,
        )
logits = dense_layer(dropout_layer, no_classes)


In [0]:
with tf.name_scope('loss'):
    with tf.device('/gpu:0'):
      softmax_cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
          labels=y_input, logits=logits)
      loss_operation = tf.reduce_mean(softmax_cross_entropy, name='loss')
    tf.summary.scalar('loss', loss_operation)

In [0]:
with tf.name_scope('optimiser'):
  with tf.device('/gpu:0'):
     optimiser = tf.train.AdagradOptimizer(learning_rate=0.02).minimize(loss_operation)

In [16]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      with tf.device('/gpu:0'):
        predictions = tf.argmax(logits, 1)
        correct_predictions = tf.equal(predictions, tf.argmax(y_input, 1))
    with tf.name_scope('accuracy'):
      with tf.device('/gpu:0'):
        accuracy_operation = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
tf.summary.scalar('accuracy', accuracy_operation)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

session = tf.Session(config = config)
session.run(tf.global_variables_initializer())

In [0]:
merged_summary_operation = tf.summary.merge_all()
train_summary_writer = tf.summary.FileWriter('./log/train', session.graph)
test_summary_writer = tf.summary.FileWriter('./log/test')

test_images, test_labels = mnist_data.test.images, mnist_data.test.labels
#valid_images, valid_labels = mnist_data. # , mnist_data.

In [0]:
for iterN in range(iter_no):
  for batch_no in range(total_batches):
      print("iter: " + str(iterN) + " - batch: " + str(batch_no) + "/" + str(total_batches))
      mnist_batch = mnist_data.train.next_batch(batch_size)
      train_images, train_labels = mnist_batch[0], mnist_batch[1]
      _, merged_summary = session.run([optimiser, merged_summary_operation],
                                      feed_dict={
                                                  x_input: train_images,
                                                  y_input: train_labels,
                                                  keep_prob : 1.0
                                                 })
      train_summary_writer.add_summary(merged_summary, (batch_no + (iterN*200)))
      if (batch_no + (iterN*200)) % 500 == 0:
          merged_summary, _ = session.run([merged_summary_operation,
                                           accuracy_operation], feed_dict={
                                                                            x_input: test_images,
                                                                            y_input: test_labels,
                                                                            keep_prob: 1.0
                                                                        })
          test_summary_writer.add_summary(merged_summary, (batch_no + (iterN*200)))